# RSA Cryptosystem

RSA stand for Rivest-Shamir-Adleman, the names of three famous computer scientists who invented the system for encrypting and decryting secret messages. The cryptosystem was discovered first by Clifford Cocks, a British mathematician. But his discovery was held as a classified state secret by the British government. 


RSA is a _public key_ cryptosystem that can encrypt and decrypt messages that one wishes to transmit across a channel. In a public key system, a user has two types of passwords or keys.
  - Public Key: $p$ is a key (or passphrase) that is announced to everyone in the world. Imagine a centralized registry where I can look up your public key or anyone else's for that matter.
  - Private/Secret Key: $s$ is a key (or passphrase) that is held secret by the user.  This key is never revealed to anyone. In fact, doing so will allow them to retrieve your secrets.
  
The idea is that if one encrypts a message using a public key, they can decrypt it with the private key or vice-versa. In other words, for any message $m$, if we encrypt it with the public key

$$ \mathsf{encrypt}(m, p) = m' $$ 

then it can be decrypted using the secret key

$$ \mathsf{decrypt}(m', s) = m$$.

In fact, this is how Alice sends a message $m$ to Bob that she does not want to reveal to anyone else:
  - Alice looks up Bob's public key from the registry: $p$.
  - She encrypts the message : $\mathsf{encrypt}(m, p) = m'$
  - She sends it to Bob who can decrypt it with his secret key (that is known only to him): $\mathsf{decrypt}(m', s) = m$.
  
However, there is a potential problem with this. Anyone can impersonate Alice and send a message in her name since they simply need to look up Bob's public key. How does Bob know that the message $m$ came from Alice and no one else?
   -  Alice prepares a special message ($A$) "This is an authentic message from Alice meant for Bob on Wed, Feb 14th and 1:28PM and the hash value of the message is 0xFA109091AB" and signs it with her secret key $s_A$. $A' = \mathsf{encrypt}(A, s_A)$.
   -  Alice appends the message $A'$ to her original message $M$.
   - She then proceeds to sign this with Bob's public key and send it to Bob.
   - Bob decrypts the message $M + A'$. 
   - To know that the message came from Alice, Bob decrypts the special encrypted part using Alice's public key: $A = \mathsf{decrypt}(A', p_A)$. Looking at $A$, Bob knows that it could have only come from Alice, since whoever signed $A'$ must have possessed Alice's private key. 
      - Also by verifying details of the message (time, hash value) in the signature, Bob knows that Alice created this signature just for the message she sent to him. I.e, Charlie did not attach Alice's signature from a message she sent him to impersonate her to Bob.
   
The ideas behind public key cryptosystems enable a lot of modern Internet commerce from securely transmitting credit card information to authenticating messages so that we know who sent the message. 

The most popular public key cryptosystem is the RSA cryptosystem. It is based on number, particularly properties of GCD, prime numbers and the difficulty of solving certain problems involving numbers. We will understand it now.


## Messages Encoded as Numbers

First, we will convert the problem of encrypting/decrypting messages in terms of operations over numbers. To do this, we have to code up messages (plain text) into a sequence of numbers that are interconvertible to the messages. One simple way to do this is to convert a given message string  as a sequence of characters using the ascii code into a bunch of numbers. 

Here are a couple of useful functions to convert a sequence of numbers into strings. We can also integrate some sort of "salt", scramble the message in some order to prevent some attacks based on frequency analysis and known/common ciphertexts but that is beyond the scope of this lecture. 

In [4]:
# given a string s, convert it into a sequence of numbers.
# It works by taking a "block length" of 5 characters and encoding the ascii values as a 
# number in the base q number sytem.
# We take q = 101 by default.
# We convert these base-q numbers into decimals.
def convert_string_to_numbers(s, q=101 ):
    # we hard code a block length of 5 for this function
    lst = [*s] # unpack string into a list
    n = len(lst)
    assert n > 0, 'Nonempty string required'
    # ord(c) for a character c provides us its ascii value
    assert all( 32 <= ord(c) <= 126 for c in lst), 'String must have alpha numeric and space characters.'
    # pad the string with a special char so that n is a multiple of block length
    r = 0 if n%5 == 0 else 5 - (n%5)
    # chr(31) is a null character that we use to pad the message to make its size a multiple of the block length (5)
    lst += [chr(31)]*r # pad it wwith a special character ascii 31
    n = len(lst) 
    assert n%5 == 0
    msg = []
    # run through the characters 5 at a time
    for i in range(0, n, 5):
        block = lst[i:i+5]
        c = [ord(k)-31 for k in block] # subtract 31 from ascii values
        m = 0
        for i in range(4, -1, -1): # convert from base-q to decimal
            m = m * q + c[i]
        msg.append(m) # append number to message
    return msg

In [5]:
msg = convert_string_to_numbers('Hello, this is a message that needs to be encoded!!')
for j in msg:
    print(j)

8404957845
7776548846
191360744
8813990599
176922693
192316710
8812885672
6973901834
7158215288
279932690
2


In [6]:
# this function inverts the process from the function for converting string to numbers
def convert_numbers_to_strings(msg, q=101 ):
    n = len(msg)
    assert n > 0, 'Nonempty list of numbers required'
    blk_len = 5 # hard coded block length
    codes = []
    for k in msg: # each number is a decimal representation of a 5 digit base q number
        for i in range(5): # convert it back into base q
            r = k%q
            if r >= 1:
                codes.append(chr(r + 31)) # convert from ascii back into character but add 31
            k = k//q
    # convert from sequence of chars back into a string
    return ''.join(codes)
    

In [7]:
convert_numbers_to_strings(msg)

'Hello, this is a message that needs to be encoded!!'

###### Basics of RSA

RSA uses modular arithmetic to encrypt/decrypt messages. 
  - Messages are natural numbers.
  - The public key is a pair of natural numbers $(e, n)$.
  - The private key is a pair of natural numbers $(d, n)$.
  - $e, d $ and $n$ will be _large numbers_. Typically, in the order of $2^{1024}$ or $2^{2048}$. These will be roughly 400 - 700 digit numbers. 

We will see soon what the properties of these keys will be soon. 

The encryption function is given by 

$$\mathsf{encrypt}(M, (e, n)) = M^e \bmod n$$

The decryption function is the same as encryption:

$$\mathsf{decrypt}(M, (d, n)) = M^d \bmod n$$


<div class="alert alert-block alert-info">
    <b> Requirement: </b> Decryption after encryption yields the original message back. Therefore, we will choose $e, d$ and $n$ so that 

$$ \mathsf{decrypt}( \mathsf{encrypt}(M, (e, n) ), (d, n) ) = (M^{e} \bmod n)^d \bmod n = M^{e \times d} \bmod n  = M \,.$$
</div>


Also, there is an added complication that constrains how we choose $e$ and $d$.

<div class="alert alert-block alert-info">
    <b> Requirement: </b> Everyone knows the public key $(e, n)$ whereas only the user knows the secret $(d, n)$. Someone who knows $e, n$  will find it very hard to work out the value of $d$.
</div>



To compute, $e, d$ satisfying the two requirements we will use a result from Euler from 1763 as a generalization of Fermat's little theorem.

<div class="alert alert-block alert-info">
    Let $n$ be a natural number. It's Euler Totient function $\varphi(n)$ is defined as the number of natural numbers $ < n$ that are relatively prime to $n$.
</div>

Recall that two natural numbers $a, b$ are relatively prime iff they have no prime factors in common, or in other words $GCD(a,b) =1$.

#### Examples 

For instance, let $n = 49$.  The set of numbers relatively prime to $n$ is the difference of the two sets $\{ 1, 2, \ldots, 49 \}  \setminus \{ 7, 14, 21, 28, 35, 42, 49 \}$. The latter includes every number that is not relatively prime to $49$. Therefore $\varphi(49) = 42 $.


Suppose $n = 15$, the numbers relatively prime to $15$ are $\{1, 2, 4, 7, 8, 11, 13, 14 \}$. Therefore, $\varphi(15) = 8$.

For any prime number $p$, $\varphi(p) = p -1$.

For any number $n$ which is the product of two primes $n = p \times q$ where $p \not= q$, we have 
$\varphi(n) = (p-1) (q-1)$. This is because, amongst the numbers $\{1, \ldots, n\}$ the set numbers that are __not__ relatively prime to $n$ include $\{p, \ldots, (q-1)p, q, \ldots, (p-1)q , pq \}$. Thus, $p + q - 1$ numbers are __not__ relatively prime. Hence, $pq - p - q + 1 = (p-1)(q-1)$ numbers are relatively prime.

### Euler's Theorem 

<div class="alert alert-block alert-warning">
Let $n$ be a natural number and $a$ be relatively prime to $n$. Then $a^{\varphi(n)}\mod n = 1$.
</div>

**Proof** Let $k = \varphi(n)$ and $S = \{ a_1, \ldots, a_k \}$ be the set of all numbers relatively prime to $n$. Note that $|S| = \varphi(n) = k$ and $a \in S$.  Consider the set 
$$S' = \{ a a_1 \bmod n,\ a a_2 \bmod n,\ \cdots,\ a a_k \bmod n \}\,.$$

Note that $S = S'$: (a) every element of $S'$ continues to be relatively prime with $n$ and therefore, $S' \subseteq S$, (b) for any two distinct elements $a_i, a_j$, $a a_i \mod n \not= a a_j \bmod n $. Therefore, $|S| = |S'|$. 
Combining, we have $ S = S'$.

Therefore, multiplying all elements of $S$ and $S'$, we have 
$$ a_1 a_2 \cdots a_k \bmod n = aa_1 a a_2 \cdots a a_k \bmod n = a^{\varphi(n)} a_1 a_2 \cdots a_k \bmod n \,.$$
Since $a, a_1, \ldots, a_k$ are all relatively prime w.r.t n, we have $a^{\varphi(n)} \bmod n = 1 \bmod n$.  __QED__

## Designing the RSA Cryptography Scheme (Take 2)

Let's get back to designing the RSA cryptography scheme. 
  - Pick two (very) large prime numbers $p, q$. 
  - Let $n = p q$.
  - Choose a number $2 \leq e < \varphi(n) $ that is relatively prime to $\varphi(n)$.
  - Since $e$ is relatively prime to $\varphi(n)$, we have $GCD(e, \varphi(n)) = 1$.
      - By Bezout's theorem, we can find integers $d, v$ so that $ e \times d - v \varphi(n) = 1$.
      - If $d$ comes out negative, we can always rewrite $d' = (\varphi(n) + d)$ and $v' = (v+e)$ so that 
      $d' > 0$, $v' > 0$ and $e\times d' - v' \times \varphi(n) =1$.
  - Choose $(e, n)$ to be the public key and $(d, n)$ to be the private key.
  - Carefully discard/forget the numbers $p, q, \varphi(n), v$ and just remember $e, d, n$.
  - Disclose $(e, n)$ as the public key while keeping $(d,n)$ secret.
  
 $e \times d = v \varphi(n) + 1$. 
 
 Let $M$ be a message. For convenience, we assume $M$ is relatively prime with $n$ (this is not a great restriction: as we will see tha for large $n$, there are relatively _few numbers_ in the range $1, \ldots, n$ that are __not__ relatively prime and the possibility that the message is one of them is very easy to avoid).
 
 We have $$\mathsf{encrypt}(M, (e, n)) = M^e \bmod n, \;\;\;\; \mathsf{decrypt}(M, (d, n)) = M^d \bmod n $$
 
 Combinbing, 
 
 $$M^{ed} \bmod n = M^{1 + v \varphi(n)} \bmod n = M \bmod n \times M^{v \varphi(n)} \mod n = M \bmod n \times 1  = M$$
 
### Example

Let us choose two prime numbers $n = 7 \times 11 = 77$.
 - We have $\varphi(n) = 60$.
 - Choose $e = 13$. We just find a number relatively prime with $60$.
 - We have $GCD(60, 13) = 1$ and the Bezout coefficients are $1 = - 23 \times 13 + 5 \times 60 $.
    - $d$ comes out negative, so we write $1 =  (60 - 23) \times 13 + (5 - 13) \times 60$.
 - We choose $d = 37$ and $v = 8$.
 

Take a message $M = 9$.

In [8]:
def encrypt_naive(M, e, n): # 
    assert 2 <= M < n
    p = 1
    for i in range(e):
        p = (p * M) % n
    return p

In [9]:
encrypt_naive(9, 13, 77) # encrypt 9 with the public key

58

In [10]:
encrypt_naive(58, 37, 77) # decrypt 9 with the private key == decryption is same as encryption for RSA just with.a different key

9

Here is the code to generate private and public keys.

In [11]:
def extended_euclid(m, n, debug=False):
    assert m >= 1 and n >= 0 and m >= n
    m0, n0 = m, n # let's rememver the initial values
    (s, t) = (1, 0)# m = s * m0 + t * n0
    (s_hat, t_hat) = (0, 1) # n = s_hat * m0 + t_hat * n0
    while n > 0:
        assert m == s * m0 + t * n0
        assert n == s_hat * m0 + t_hat * n0
        q = m // n # compute the integer quotient 
        r = m % n # the reminder 
        # m = q * n + r, or alternatively, r = m - q * n = (s-q*s_hat) * m_0 + (t - q * t_hat)* n_0  
        a, b = (s - q*s_hat, t - q * t_hat)
        (m, n, s, t, s_hat, t_hat) = (n, r, s_hat, t_hat, a, b)
        if debug:
            print(f'GCD({m0}, {n0}) = GCD({m}, {n})')
            print(f'\t {m} = {s}*{m0} + {t} * {n0}')
            print(f'\t {n} = {s_hat}*{m0} + {t_hat} * {n0}')
    return m, s, t
        

In [12]:
from random import randint
def generate_rsa_keys(p, q):
    assert p < q
    n = p * q
    phi = (p-1)*(q-1)
    e = None
    d = None
    for e in range(p//3, p):
        (i, s, t) = extended_euclid(phi, e)
        if i == 1:
            if t < 0:
                d = phi + t
            else:
                d = t
            break
    return (n, e, d)

    
    

In [13]:
generate_rsa_keys(7, 13)

(91, 5, 29)

The implementation of modular exponentiation is naive. It takes as much time as the number $e$ which can be very large. Thankfully, we can do modular exponentiation to calculate $M^e \bmod n$ in time linear in $\log(e)$, the number of bits in the number $e$.

The idea is to view the bit representation of $e$ as 
$$ e = e_k e_{k-1} \ldots e_0 = e_0 + 2 e_1 + 2^2 e_2 + \cdots + 2^{k} e_k $$

Therefore, $M^{e} = M^{e_k} M^{2e_1} \cdots M^{2^i e_i} \cdots M^{2^k e_k}$. 

We compute by repeated squaring, 
$$ M \bmod n, M^2 \bmod n, M^{2^2} \bmod n, M^{2^3} \bmod n, \cdots $$

If $e_i = 1$, then we multiply the final result by $M^{2^i}$.

In [82]:
def modular_exp(M, e, n):
    p = 1 # this is the final result
    exp = M # this will be the exponent M^{2^i}
    while e > 0:
        b = e % 2 # extract least significant bit 
        if b == 1:
            p = (p * exp)%n
        exp = (exp * exp)%n # exp is now M^{2^{i+1}}
        e = e // 2 # integer division
    return p

In [83]:
modular_exp(9, 13, 77) # encrypt 9 with the public key

58

In [84]:
modular_exp(58, 37, 77) # encrypt 9 with the public key

9

Note that our messages are roughly 10 digit numbers. We need to find prime numbers $p, q$ so that $M < p, M < q$. This guarantees that $M$ will be relatively prime w.r.t $\varphi(n)$. Let's choose $p = 735193$ and $q = 875491$.

In [104]:
(n, e, d) = generate_rsa_keys(735193, 875491)
print(f'Public Key: {e, n}')
print(f'Private Key: {d, n}')

Public Key: (245071, 643654854763)
Private Key: (478235009071, 643654854763)


In [105]:
def rsa_encode(msg_list, e, n):
    rsa_lst = [modular_exp(M, e, n) for M in msg_list]
    return rsa_lst

In [106]:
orig_msg = convert_string_to_numbers('Hello, I have to say something secret!!')
print('Original message:')
print(orig_msg)
print(f'Public Key: {e, n}')
msg_rsa = rsa_encode(orig_msg, e, n)
print('RSA encoded')
print(msg_rsa)

Original message:
[8404957845, 7597868130, 8846887309, 9434293021, 7365416113, 7574504983, 8707796306, 2089659]
Public Key: (245071, 643654854763)
RSA encoded
[46264071735, 544973177667, 185197985834, 152593409466, 416635991589, 482835244299, 65259378513, 220105851236]


In [109]:
# If we tried to read the encrypted message, we get junk back.
print('RSA encoded string is')
convert_numbers_to_strings(msg_rsa)


RSA encoded string is


"2cLZG>>['u)FAg]=`aFS]ann_98,\x7f}T(*,4r_X0~"

In [110]:
msg_decode = rsa_encode(msg_rsa, d, n)
print('Decrypted message')
print(msg_decode)
convert_numbers_to_strings(msg_decode)

Decrypted message
[8404957845, 7597868130, 8846887309, 9434293021, 7365416113, 7574504983, 8707796306, 2089659]


'Hello, I have to say something secret!!'

## Summary of RSA

Note that RSA depends on finding two large prime numbers $p, q$ wherein $n = pq$. 
We then generate the key by choosing a number $e$ relatively prime with $n$ and $\varphi(n) = (p-1)(q-1)$.
Using extended Euclid's algorithm yields us the decryption key $d$. 

We reveal $(e, n)$ to the world and keep $(d, n)$ as a secret.

We get that a message encrypted with the public key is decrypted with the private key.

## Security of RSA

The security of a public-key cryptosystem depends on the hardness of finding the secret key just knowing private key. How does RSA fare in this respect?

Suppose we known $(e, n)$, can we guess the private key $(d, n)$? As it turns out, this is equivalent to finding the factors $p,q $ of  $n$.
  - If we know $p, q$, we can always run through the same process with $e$, the Euclidean GCD algorithm to find $d$.
  - If we knew $d$ then, we know that $e \times d = 1 + v \varphi(n)$. Therefore, knowing $e, d$, we can find $\varphi(n) = (p-1)(q-1)$. Knowing $n, \varphi(n)$, we can find $p, q$.
  
  
 
 
<div class="alert alert-block alert-warning">
 
 Therefore, breaking RSA security is computationally equivalent to factoring a number $n$ that is known to be the product of two prime numbers. Such numbers are called <i> semi-prime numbers</i>.  We do not know if there is an efficient algorithm to factor semi-prime numbers. Here efficient means an algorithm that runs in time polynomial in the number of bits of $n$
</div>

Does this mean that no such algorithm exists? This remains an open question in computer science. We know that factoring numbers is a hard problem and 50 years of efforts have not yielded algorithms that are practical on current computers. However, note that there are known efficient algorithms for factoring numbers on a _quantum computer_. In fact, this is the topic we will explore next.

## Finding Prime Numbers

We will briefly talk about how to find large prime numbers. This involves generating numbers starting from a randomly chosen odd number and testing if the generated numbers are prime. The most practical algorithm for primality testing is called the _Rabin-Miller Primality Test_ It runs very fast but has a small probability of accepting a number as prime when it is actually not a prime. This probability can be made as small as one wishes by repeating the test. The _Aggarwal-Khayal-Saxena_ (AKS) test is a deterministic test that is theoretically efficient and does not rely on randomness. But to our knowledge it is not used in practical implementations of RSA. 
  
  
